<a href="https://colab.research.google.com/github/lizhihang1017/SourceCode/blob/main/GraphData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 数据预处理

In [1]:
import pandas as pd

# 读取数据
data = pd.read_csv('testdata.txt', sep=' ', header=None, usecols=[0, 1, 6], names=['node1', 'node2', 'relation'])

print(data.head())

   node1  node2 relation
0  40609  40611        m
1  44818  44819        m
2  46346  46349        m
3  15220  15222        m
4  25624  25626        m


## 1.1 缺失值、重复值处理

In [2]:
# 检查缺失值
print(data.isnull().sum())

# 去除包含缺失值的行
data = data.dropna()

node1       0
node2       0
relation    0
dtype: int64


In [3]:
# prompt: data 的数量

print("去重之前的数据量：%d" % len(data))
data = data.drop_duplicates() # 去重
print("去重之后的数据量：%d" % len(data))

去重之前的数据量：76505
去重之后的数据量：76505


##1.2 配偶关系异常处理

In [4]:
# 构建家庭树
def build_family_tree(relationships):
    family_tree = defaultdict(lambda: {'children': set(), 'spouse': None})

    for person1, person2, relation in relationships:
        if relation in ['f', 'm']:
            family_tree[person1]['children'].add(person2)
        elif relation == 's':
            family_tree[person1]['spouse'] = person2
            family_tree[person2]['spouse'] = person1

    return family_tree

In [5]:
# 验证关系的合理性
def validate_relationships(family_tree):
    invalid_entries = []

    for person, relations in family_tree.items():
        children = relations['children']
        spouse = relations['spouse']

        # 检查配偶不应是子女
        if spouse is not None:
            for child in children:
                if child == spouse:
                    invalid_entries.append((person, child, '配偶是子女'))

            # 检查子女不应是配偶
            if spouse in children:
                invalid_entries.append((person, spouse, '子女是配偶'))

            # 检查配偶不应是父母、孙子、孙女
            for child in children:
                if child in family_tree:
                    grandchildren = family_tree[child]['children']
                    if spouse in grandchildren:
                        invalid_entries.append((person, spouse, '配偶是孙子孙女'))

    return invalid_entries

In [6]:
# 保存无效关系到文件
def save_invalid_relationships(invalid_entries, output_file):
    with open(output_file, 'w') as file:
        for entry in invalid_entries:
            file.write(f"无效关系: {entry[0]} 和 {entry[1]} 的关系是无效的，因为{entry[2]}。\n")

In [8]:
# 读取数据
def load_data(file_path):
    relationships = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 5:
                person1 = parts[0]
                person2 = parts[1]
                relation = parts[-1]
                relationships.append((person1, person2, relation))
    return relationships

In [9]:
from collections import defaultdict

# 主程序
input_file = 'testdata.txt'
output_file = 'invalid_relationships.txt'
relationships = load_data(input_file)

family_tree = build_family_tree(relationships)
invalid_relationships = validate_relationships(family_tree)

# 输出无效关系并保存到文件
if invalid_relationships:
    # for entry in invalid_relationships:
    #     print(f"无效关系: {entry[0]} 和 {entry[1]} 的关系是无效的，因为{entry[2]}。")

    save_invalid_relationships(invalid_relationships, output_file)
    print(f"无效关系已保存到 {output_file}。")
else:
    print("没有发现不合理的家庭关系。")


无效关系已保存到 invalid_relationships.txt。


In [13]:
from collections import defaultdict

'''
过滤掉无效信息
'''

# 读取无效关系数据
def load_invalid_relationships(file_path):
    invalid_pairs = set()
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) >= 4:
                person1 = parts[1]
                person2 = parts[3]
                invalid_pairs.add((person1, person2))
                invalid_pairs.add((person2, person1))  # 考虑配偶关系是双向的
    return invalid_pairs

# 读取有效数据
def load_data(file_path):
    relationships = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 5:
                person1 = parts[0]
                person2 = parts[1]
                relation = parts[-1]
                relationships.append((person1, person2, relation))
    return relationships

# 过滤无效配偶关系
def filter_valid_relationships(relationships, invalid_pairs):
    valid_relationships = []
    for person1, person2, relation in relationships:
        if relation == 's':
            if (person1, person2) not in invalid_pairs:
                valid_relationships.append((person1, person2, relation))
        else:
            valid_relationships.append((person1, person2, relation))
    return valid_relationships

# 保存有效关系到文件
def save_cleaned_data(valid_relationships, output_file):
    with open(output_file, 'w') as file:
        for person1, person2, relation in valid_relationships:
            file.write(f"{person1} {person2}    {relation}\n")

# 主程序
invalid_file = 'invalid_relationships.txt'
input_file = 'testdata.txt'
output_file = 'cleaned_testdata.txt'

# 读取无效关系和有效数据
invalid_relationships = load_invalid_relationships(invalid_file)
relationships = load_data(input_file)

# 过滤有效关系
valid_relationships = filter_valid_relationships(relationships, invalid_relationships)

# 保存有效关系到文件
save_cleaned_data(valid_relationships, output_file)

print(f"清洁后的数据已保存到 {output_file}。")

清洁后的数据已保存到 cleaned_testdata.txt。


## 1.3 多角色节点处理

In [14]:
# 构建家庭树并统计角色
def build_family_tree(relationships):
    family_tree = defaultdict(lambda: {'fathers': 0, 'mothers': 0, 'father_nodes': [], 'mother_nodes': []})

    # 统计的是当父亲或当母亲的次数！！！！
    for person1, person2, relation in relationships:
        if relation == 'f':
            family_tree[person1]['fathers'] += 1
            family_tree[person1]['father_nodes'].append(person2)
        elif relation == 'm':
            family_tree[person1]['mothers'] += 1
            family_tree[person1]['mother_nodes'].append(person2)

    return family_tree

In [15]:
# 修改不一致的角色，以下"父亲数量"均为当前节点当父亲的次数，母亲数量 同理
def resolve_roles(family_tree):
    modified_relationships = []

    for person, roles in family_tree.items():
        father_count = roles['fathers']
        mother_count = roles['mothers']

        # 如果父亲数量大于母亲数量，修改母亲角色为父亲
        if father_count > mother_count and mother_count != 0:
            for mother in roles['mother_nodes']:
                modified_relationships.append((person, mother, 'f'))  # 修改为父亲关系

        # 如果母亲数量大于父亲数量，修改父亲角色为母亲
        elif mother_count > father_count and father_count != 0:
            for father in roles['father_nodes']:
                modified_relationships.append((person, father, 'm'))  # 修改为母亲关系

        # 如果父亲和母亲数量相同，依照顺序修改
        elif father_count == mother_count and father_count > 0:
            # 获取父亲和母亲节点的出现顺序
            first_father = roles['father_nodes'][0] if roles['father_nodes'] else None
            first_mother = roles['mother_nodes'][0] if roles['mother_nodes'] else None

            # 根据顺序修改关系
            if first_father:
                # 将所有母亲关系修改为父亲关系
                for mother in roles['mother_nodes']:
                    modified_relationships.append((person, mother, 'f'))  # 修改为父亲关系
            elif first_mother:
                # 将所有父亲关系修改为母亲关系
                for father in roles['father_nodes']:
                    modified_relationships.append((person, father, 'm'))  # 修改为母亲关系

    return modified_relationships

In [16]:
# 读取数据
def load_data(file_path):
    relationships = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 5 or True:
                person1 = parts[0]
                person2 = parts[1]
                relation = parts[-1]
                relationships.append((person1, person2, relation))
    return relationships


# 保存修改的关系到文件
def save_modifications(modified_relationships, output_file):
    with open(output_file, 'w') as file:
        for entry in modified_relationships:
            file.write(f"修改关系: {entry[0]} 和 {entry[1]} 的关系修改为 {entry[2]}\n")

In [17]:
# 主程序
input_file = 'cleaned_testdata.txt'
output_file = 'modifications_log.txt'
relationships = load_data(input_file)
family_tree = build_family_tree(relationships)
modified_relationships = resolve_roles(family_tree)

# 输出并保存修改的关系
if modified_relationships:
    save_modifications(modified_relationships, output_file)
    print(f"修改的关系已保存到文件 {output_file}。")
else:
    print("没有需要修改的关系。")


修改的关系已保存到文件 modifications_log.txt。


In [18]:
# 读取数据集
def load_data(file_path):
    relationships = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 5 or True:
                person1 = parts[0]
                person2 = parts[1]
                relation = parts[-1]
                relationships.append((person1, person2, relation))
    return relationships

# 读取修改日志文件
def load_modifications(modifications_file):
    modifications = []
    with open(modifications_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 4 or True:  # 检查每行是否符合预期格式
                person1 = parts[1]
                person2 = parts[3]
                new_relation = parts[-1].replace('关系修改为', '')  # 去掉描述性文字
                modifications.append((person1, person2, new_relation))

    return modifications

# 根据修改日志更新数据集
def apply_modifications(original_data, modifications):
    modified_data = original_data[:]
    for person1, person2, new_relation in modifications:
        for i, (orig_person1, orig_person2, orig_relation) in enumerate(modified_data):
            if orig_person1 == person1 and orig_person2 == person2:
                modified_data[i] = (person1, person2, new_relation)  # 更新关系
                break
    return modified_data

# 保存修改后的数据集
def save_data(data, file_path):
    with open(file_path, 'w') as file:
        for person1, person2, relation in data:
            file.write(f"{person1} {person2} 1 l {relation}\n")

# 主程序
input_file = 'cleaned_testdata.txt'
modifications_file = 'modifications_log.txt'
output_file = 'cleaned_testdata.txt'

# 加载原始数据和修改日志
relationships = load_data(input_file)
modifications = load_modifications(modifications_file)

# 应用修改并保存清洁的数据
cleaned_data = apply_modifications(relationships, modifications)
save_data(cleaned_data, output_file)

print(f"数据已更新并保存到 {output_file}")


数据已更新并保存到 cleaned_testdata.txt


#2. 数据分析

## 2.1 最长路径查询

In [23]:
import networkx as nx
import time

'''
通过动态规划结合拓扑排序计算有向图中的最长路径。
首先对图进行拓扑排序，以确保节点按依赖关系顺序处理。算法将每个节点的最长路径长度初始化为0，并记录其前驱节点。
随后，依次遍历排序后的节点，逐步更新每个后继节点的最长路径长度，确保始终保持路径最大。
最后，通过选择路径长度最大的终点，回溯前驱节点还原出最长路径，并记录其长度。
'''


# 创建一个空的有向图
G = nx.DiGraph()

print("开始读取数据文件并构建图...")
# 读取数据文件
with open('cleaned_testdata.txt', 'r') as file:
    lines = file.readlines()

# 解析每一行数据并构建图
for line in lines:
    parts = line.strip().split()
    if len(parts) >= 4 or True:
        person1_id = str(parts[0])  # 确保ID为字符串
        person2_id = str(parts[1])  # 确保ID为字符串
        relation_type = parts[-1]

        # 添加边并设置关系类型作为边的属性
        G.add_edge(person1_id, person2_id, type=relation_type)

print("图构建完成！")

# 环检测
if nx.is_directed_acyclic_graph(G):
    print("图中无环，开始计算最长路径...")
else:
    print("图中存在环，无法计算拓扑排序。请检查数据文件！")
    exit(1)

# 定义最长路径保存文件的路径
output_file = "longest_path.txt"

def save_longest_path(path, length):
    """实时保存最长路径和长度到本地文件."""
    with open(output_file, "w") as f:
        f.write(f"最长路径长度: {length}\n")
        f.write("路径: " + str(path) + "\n")

def find_longest_path(graph):
    print("开始计算最长路径...")

    # 拓扑排序
    topological_order = list(nx.topological_sort(graph))

    # 初始化最长路径和前驱节点字典
    longest_path = {node: 0 for node in graph.nodes}
    predecessor = {node: None for node in graph.nodes}

    # 动态规划计算最长路径
    for node in topological_order:
        for predecessor_node in graph.predecessors(node):
            if longest_path[predecessor_node] + 1 > longest_path[node]:
                longest_path[node] = longest_path[predecessor_node] + 1
                predecessor[node] = predecessor_node

    # 找到最长路径的终点
    end_node = max(longest_path, key=longest_path.get)
    max_length = longest_path[end_node]

    # 回溯找到最长路径
    path = []
    current_node = end_node
    while current_node is not None:
        path.append(current_node)
        current_node = predecessor[current_node]

    # 反转路径以获得正确的顺序
    path = path[::-1]

    print(f"找到最长路径：长度为 {max_length}，路径为 {path}")
    save_longest_path(path, max_length)

    return path, max_length

# 记录程序开始时间
start_time = time.time()
print("程序开始执行...")

# 查找最长路径
longest_path, max_length = find_longest_path(G)

# 计算并显示总运行时间
end_time = time.time()
total_time = end_time - start_time
print(f"程序总运行时间: {total_time:.2f} 秒")

print(f"最终最长路径长度为: {max_length}")
print(f"路径为: {longest_path}")
print("程序执行完毕。")

开始读取数据文件并构建图...
图构建完成！
图中无环，开始计算最长路径...
程序开始执行...
开始计算最长路径...
找到最长路径：长度为 14，路径为 ['9687', '9688', '9691', '10261', '15258', '15259', '15948', '23446', '23447', '23467', '23476', '23478', '23502', '23506', '23555']
程序总运行时间: 0.35 秒
最终最长路径长度为: 14
路径为: ['9687', '9688', '9691', '10261', '15258', '15259', '15948', '23446', '23447', '23467', '23476', '23478', '23502', '23506', '23555']
程序执行完毕。


## 2.2 社区检测 最大家庭族谱

In [20]:
import networkx as nx
import community.community_louvain as community_louvain
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: 读取数据并构建有向图
G = nx.DiGraph()
print("开始读取数据文件并构建图...")
with open('cleaned_testdata.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 3 or True:
            person1_id = str(parts[0])  # 确保ID为字符串
            person2_id = str(parts[1])  # 确保ID为字符串
            relation_type = parts[-1]

            # 添加边并设置关系类型作为边的属性
            G.add_edge(person1_id, person2_id, type=relation_type)

print("图构建完成！")

# Step 2: 将有向图转换为无向图用于社区检测
G_undirected = G.to_undirected()

# Step 3: 使用 Louvain 方法进行社区检测
print("开始进行社区检测...")
partition = community_louvain.best_partition(G_undirected)

# Step 4: 找到最大社区
# 统计每个社区的节点数量
community_sizes = {}
for node, community_id in partition.items():
    if community_id not in community_sizes:
        community_sizes[community_id] = 0
    community_sizes[community_id] += 1

# 找到最大社区的 ID
max_community_id = max(community_sizes, key=community_sizes.get)
print(f"最大的社区ID是 {max_community_id}，包含 {community_sizes[max_community_id]} 个节点。")

# 获取最大社区中的节点
max_community_nodes = [node for node, community_id in partition.items() if community_id == max_community_id]

# Step 5: 提取最大社区的子图并导出
max_community_subgraph = G.subgraph(max_community_nodes).copy()

# 将最大社区的节点和边导出到 CSV 文件
nodes_data = {'Node': list(max_community_subgraph.nodes())}
edges_data = {
    'Source': [edge[0] for edge in max_community_subgraph.edges()],
    'Target': [edge[1] for edge in max_community_subgraph.edges()],
    'Relation': [max_community_subgraph.edges[edge]['type'] for edge in max_community_subgraph.edges()]
}

nodes_df = pd.DataFrame(nodes_data)
edges_df = pd.DataFrame(edges_data)

# 导出为CSV文件
nodes_df.to_csv("max_community_nodes.csv", index=False)
edges_df.to_csv("max_community_edges.csv", index=False)

print("最大社区的节点和边已保存到 'max_community_nodes.csv' 和 'max_community_edges.csv'")


开始读取数据文件并构建图...
图构建完成！
开始进行社区检测...
最大的社区ID是 153，包含 342 个节点。
最大社区的节点和边已保存到 'max_community_nodes.csv' 和 'max_community_edges.csv'


In [21]:
import networkx as nx
import pandas as pd

# 读取最大社区的边数据
print("读取最大社区边数据文件...")
edges_df = pd.read_csv("max_community_edges.csv", header=None, names=["person1_id", "person2_id", "relation_type"])

# 创建有向图
G = nx.DiGraph()
print("开始构建有向图...")

# 添加边到图中
for _, row in edges_df.iterrows():
    G.add_edge(row["person1_id"], row["person2_id"], relation=row["relation_type"])

print("图构建完成！")

# 计算弱连通分量
print("开始检测弱连通分量...")
weakly_connected_components = list(nx.weakly_connected_components(G))

# 统计弱连通分量的数量和规模
num_components = len(weakly_connected_components)
largest_component = max(weakly_connected_components, key=len)

print(f"弱连通分量的数量: {num_components}")
print(f"最大的弱连通分量节点数: {len(largest_component)}")

# 如果仅有一个分量，则整个图为最大家庭族谱
if num_components == 1:
    print("数据中只有一个弱连通分量，即为最大家庭族谱。")
else:
    print("数据包含多个弱连通分量，每个分量代表独立的家庭或家庭群体。")
    print("最大家庭族谱的节点集合为: ")
    print(largest_component)

# 将最大的弱连通分量子图保存到新文件
print("提取并保存最大家庭族谱的数据...")
largest_subgraph = G.subgraph(largest_component)

# 将边数据导出为 CSV
output_edges = []
for u, v, data in largest_subgraph.edges(data=True):
    output_edges.append([u, v, data["relation"]])

# 转换为 DataFrame 并保存
largest_edges_df = pd.DataFrame(output_edges, columns=["person1_id", "person2_id", "relation_type"])
largest_edges_df.to_csv("largest_family_ancestry_edges.csv", index=False)

print("最大家庭族谱边数据已保存到 'largest_family_ancestry_edges.csv'")

读取最大社区边数据文件...
开始构建有向图...
图构建完成！
开始检测弱连通分量...
弱连通分量的数量: 2
最大的弱连通分量节点数: 342
数据包含多个弱连通分量，每个分量代表独立的家庭或家庭群体。
最大家庭族谱的节点集合为: 
{'5572', '2196', '48462', '46116', '48660', '37076', '48308', '48371', '48297', '48806', '48692', '2180', '48436', '47207', '48564', '48790', '48380', '47554', '6181', '48609', '48472', '46154', '48435', '48403', '48607', '48049', '48437', '48549', '48369', '47830', '46380', '45684', '48656', '46406', '48608', '48698', '37100', '48810', '48471', '48575', '46078', '48512', '48685', '48452', '46111', '48651', '48507', '46407', '46079', '45682', '48513', '48451', '46156', '48646', '48330', '48340', '2179', '48171', '48805', '48567', '6137', '48306', '48461', '48803', '48649', '36960', '48684', '48402', '48674', '36796', '47573', '48638', '48761', '48370', '48309', '6162', '48782', '48606', '48493', '48486', '48488', '37099', '5633', '48494', '6203', '48543', '46461', '46384', '48374', '45777', '48327', '48539', '48570', '48762', '45683', '48307', '48635', '9', '48689', '

In [22]:
import pandas as pd
from graphviz import Digraph

# 读取家庭关系数据
family_edges = pd.read_csv('largest_family_ancestry_edges.csv')

# 创建 Graphviz 有向图
dot = Digraph(comment='Family Ancestry', format='svg')
dot.attr(rankdir='LR')  # 设置为横向

# 添加节点和边
for _, row in family_edges.iterrows():
    person1_id = str(row['person1_id'])
    person2_id = str(row['person2_id'])
    relation = row['relation_type']

    # 添加节点
    dot.node(person1_id, person1_id)
    dot.node(person2_id, person2_id)

    # 添加边
    if relation == 'm':
        dot.edge(person1_id, person2_id, label='mother', color='blue')
    elif relation == 'f':
        dot.edge(person1_id, person2_id, label='father', color='green')
    elif relation == 's':
        dot.edge(person1_id, person2_id, label='Spouse', color='red')

# 保存为 SVG 文件
dot.render('family_ancestry_horizontal', view=True)

'family_ancestry_horizontal.svg'